In [96]:
from datetime import datetime

#from metpy.calc.indices import precipitable_water
from metpy.calc.tools import get_layer
from metpy.calc import (mixing_ratio, saturation_vapor_pressure, get_wind_dir, log_interp, equivalent_potential_temperature,
                        parcel_profile, cape_cin, saturation_mixing_ratio)
from metpy.io import get_upper_air_data
from metpy.io.upperair import UseSampleData
from metpy.testing import assert_array_equal, assert_almost_equal
from metpy.units import units
import numpy as np
import numpy.ma as ma
from metpy.constants import g, rho_l

In [97]:
with UseSampleData():
    data = get_upper_air_data(datetime(2016, 5, 22, 0), 'DDC', source='wyoming')

In [98]:
print(data.variables)

OrderedDict([('pressure', <metpy.io._tools.UnitLinker object at 0x000001E16DBC9198>), ('height', <metpy.io._tools.UnitLinker object at 0x000001E16DBC9FD0>), ('temperature', <metpy.io._tools.UnitLinker object at 0x000001E16DBC9780>), ('dewpoint', <metpy.io._tools.UnitLinker object at 0x000001E16DBC9E80>), ('u_wind', <metpy.io._tools.UnitLinker object at 0x000001E16DBC9F28>), ('v_wind', <metpy.io._tools.UnitLinker object at 0x000001E16DAD44E0>), ('speed', <metpy.io._tools.UnitLinker object at 0x000001E16DAD4278>), ('direction', <metpy.io._tools.UnitLinker object at 0x000001E16DAD4080>)])


In [99]:
def most_unstable_parcel(pressure, temperature, dewpoint, heights=None,
                         bottom=None, depth=300 * units.hPa):
    """
    Determine the most unstable parcel in a layer.
    Determines the most unstable parcle of air by calculating the equivalent
    potential temperature and finding its maximum in the specified layer.
    Parameters
    ----------
    pressure: `pint.Quantity`
        Atmospheric pressure profile
    temperature: `pint.Quantity`
        Atmospheric temperature profile
    dewpoint: `pint.Quantity`
        Atmospheric dewpoint profile
    heights: `pint.Quantity`, optional
        Atmospheric height profile. Standard atmosphere assumed when None (the default).
    bottom: `pint.Quantity`, optional
        Bottom of the layer to consider for the calculation in pressure or height.
        Defaults to using the bottom pressure or height.
    depth: `pint.Quantity`, optional
        Depth of the layer to consider for the calculation in pressure or height. Defaults
        to 300 hPa.
    Returns
    -------
    `pint.Quantity`
        Pressure, temperature, and dew point of most unstable parcel in the profile.
    See Also
    --------
    get_layer
    """
    p_layer, T_layer, Td_layer = get_layer(pressure, temperature, dewpoint, bottom=bottom,
                                           depth=depth, heights=heights, interpolate=False)
    theta_e = equivalent_potential_temperature(p_layer, T_layer, Td_layer)
    max_idx = np.argmax(theta_e)
    return p_layer[max_idx], T_layer[max_idx], Td_layer[max_idx], max_idx

In [100]:
def mucape_cin(pressure, temperature, dewpoint, **kwargs):
    r"""Calculate MUCAPE and MUCIN.
    Calculate the convective available potential energy (CAPE) and convective inhibition (CIN)
    of a given upper air profile and most unstable parcel path. CIN is integrated between the
    surface and LFC, CAPE is integrated between the LFC and EL (or top of sounding).
    Intersection points of the measured temperature profile and parcel profile are linearly
    interpolated.
    Parameters
    ----------
    pressure : `pint.Quantity`
        The atmospheric pressure level(s) of interest. The first entry should be the starting
        point pressure.
    temperature : `pint.Quantity`
        The sounding temperature
    dewpoint : `pint.Quantity`
        The sounding dew point
    Returns
    -------
    `pint.Quantity`
        Convective available potential energy (CAPE) for the most unstable parcel.
    `pint.Quantity`
        Convective inhibition (CIN) for the most unstable parcel.
    See Also
    --------
    cape_cin, lfc, el
    """
    mu_parcel = most_unstable_parcel(pressure, temperature, dewpoint, **kwargs)
    max_idx = mu_parcel[3]
    mu_pressure_profile = pressure[max_idx:]
    mu_temperature_profile = temperature[max_idx:]
    mu_dewpoint_profile = dewpoint[max_idx:]
    mu_profile = parcel_profile(mu_pressure_profile, mu_parcel[1], mu_parcel[2])
    return cape_cin(mu_pressure_profile, mu_temperature_profile,
                    mu_dewpoint_profile, mu_profile)

In [101]:
def significant_hail_parameter(pressure, u, v, heights, temperature, dewpoint):
    """Calculate the significant hail parameter.
    """
    # Take the guts of most unstable cape, since I'd be calling the most_unstable_parcel
    # function twice if I actually used most_unstable_cape
    mu_parcel = most_unstable_parcel(pressure, temperature, dewpoint)
    max_idx = mu_parcel[3]
    mu_pressure_profile = pressure[max_idx:]
    mu_temperature_profile = temperature[max_idx:]
    mu_dewpoint_profile = dewpoint[max_idx:]
    mu_profile = parcel_profile(mu_pressure_profile, mu_parcel[1], mu_parcel[2])
    mucape, _ = cape_cin(mu_pressure_profile, mu_temperature_profile,
                         mu_dewpoint_profile, mu_profile)
    print(mucape)
    #Get most unstable parcel mixing ratio
    mu_mixr = saturation_mixing_ratio(mu_pressure_profile[0], mu_dewpoint_profile[0]).to('g/kg')
    print(mu_mixr.to('g/kg'))
    layer_75p, layer_75t, layer_75h = get_layer(pressure, temperature, heights, bottom=700 * units('hPa'), depth=200 * units('hPa'))
    lapse_rate = (layer_75t[0] - layer_75t[-1]) / ((layer_75h[0] - layer_75h[-1]).to('kilometers'))
    print(lapse_rate)
    temp_500 = temperature[pressure == 500 * units('hPa')]
    print(temp_500)
    _, _, shear6 = bulk_shear(pressure, u, v, heights=heights, depth=6000 * units('meter'))
    print(shear6)
    freezing_level = heights[temperature < 0 * units('degC')][0]-heights[0]
    print(freezing_level)
    
    

In [102]:
significant_hail_parameter(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], data.variables['height'][:], data.variables['temperature'][:], data.variables['dewpoint'][:])

2497.603714056522 joule / kilogram
13.677279366001597 gram / kilogram
-7.566157286619454 delta_degC / kilometer
[-10.1] degC
17.07021536290158 meter / second
3477.0 meter


In [103]:
def bunkers_storm_motion(pressure, u, v, heights):
    r"""Calculate the Bunkers RM and LM storm motions and sfc-6km mean flow.
    
    Uses the storm motion calculation from [Bunkers et al, 2000]_,
    which does not require effective inflow layer calculations,
    unlike the more recent version in [Bunkers et al 2014]_.
    
    Parameters
    ----------
    pressure : array-like
        Pressure from sounding
    u : array-like
        U component of the wind
    v : array-like
        V component of the wind
    heigts : array-like
        Heights from sounding
    
    Returns
    -------
    RM_vector
        U and v component of Bunkers RM storm motion
    LM_vector
        U and v component of Bunkers LM storm motion
    mean_vector
        U and v component of sfc-6km mean flow
        
    See Also
    --------
    bunkers_storm_motion_2014
    
    Citations:
    
    Bunkers, M. J., B. A. Klimowski, J. W. Zeitler, R. L. Thompson, and M. L. Weisman, 2000:
        Predicting supercell motion using a new hodograph technique. Wea. Forecasting, 15, 61–79.
    
    
    Bunkers, M. J., D. A. Barber, R. L. Thompson, R. Edwards, and J. Garner, 2014: 
        Choosing a universal mean wind for supercell motion prediction. J. Operational Meteor.,
        2 (11), 115–129, doi: https://dx.doi.org/10.15191/nwajom.2014.0211.
    
    
     """

    ums = u.to('m/s')
    vms = v.to('m/s')
    
    
    # mean wind from sfc-6km
    u6m, v6m = mean_pressure_weighted(pressure, ums, vms, heights=heights, depth = 6000 * units('meter'))

    # mean wind from sfc-500m
    u5m, v5m = mean_pressure_weighted(pressure, ums, vms, heights=heights, depth = 500 * units('meter'))
    
    # mean wind from 5.5-6km
    u55m, v55m = mean_pressure_weighted(pressure, ums, vms, heights=heights, depth = 500 * units('meter'), 
                                             bottom = heights[0] + 5500 * units('meter'))

    # Calculate the shear vector from sfc-500m to 5.5-6km
    u6shr = u55m - u5m
    v6shr = v55m - v5m

    # making the shear vector
    vl = [u6shr.magnitude, v6shr.magnitude, 0]

    # Create a k hat vector
    vk = [0,0,1]

    # Take the cross product of the wind shear and k, and divide by the vector magnitude and 
    # multiply by the deviaton empirically calculated in Bunkers (2000) (7.5 m/s)
    rdev = np.cross(vl, vk) * (7.5 / (np.sqrt(u6shr.magnitude ** 2 + v6shr.magnitude ** 2)))
    ldev = np.cross(vk, vl) * (7.5 / (np.sqrt(u6shr.magnitude ** 2 + v6shr.magnitude ** 2)))
    
    # Add the deviations to the layer average wind to get the RM motion
    uRM = u6m.magnitude + rdev[0]
    vRM = v6m.magnitude + rdev[1]

    # Subtract the deviations to get the LM motion
    uLM = u6m.magnitude - rdev[0]
    vLM = v6m.magnitude - rdev[1]
    
    RM_vector = np.asarray([uRM, vRM]) * units('m/s')
    
    LM_vector = np.asarray([uLM, vLM]) * units('m/s')
    
    mean_vector = np.asarray([u6m.magnitude, v6m.magnitude]) * units('m/s').to(u.units)
    
    return RM_vector, LM_vector, mean_vector




In [104]:
def bulk_shear(pressure, u, v, heights=None, bottom=None, depth=None):
    r"""Calculate bulk shear through a layer. 
    
    Layer top and bottom specified in meters or pressure.
    
    Parameters
    ----------
    pressure : `pint.Quantity`
        Atmospheric pressure profile
    u : `pint.Quantity`
        U-component of wind.
    v : `pint.Quantity`
        V-component of wind.
    height : `pint.Quantity`
        Heights from sounding
    depth: `pint.Quantity`
        The depth of the layer in meters or hPa
    bottom: `pint.Quantity`
        The bottom of the layer in meters or hPa.
        If in meters, must be in the same coordinates as the given
        heights (i.e., don't use meters AGL unless given heights
        are in meters AGL.) Default is the surface (1st observation.)
        
    Returns
    -------
    `pint.Quantity'
        u_shr: u-component of layer bulk shear, in m/s
    `pint.Quantity'
        v_shr: v-component of layer bulk shear, in m/s
    `pint.Quantity'
        shr_mag: magnitude of layer bulk shear, in m/s
        
    """   
    
    
    u = u.to('meters/second')
    v = v.to('meters/second')
    
    sort_inds = np.argsort(pressure[::-1])
    pressure = pressure[sort_inds]
    heights = heights[sort_inds]
    u = u[sort_inds]
    v = v[sort_inds]
        
    w_int = get_layer(pressure, u, v, heights=heights, bottom=bottom, depth=depth)
    
    u_shr = w_int[1][-1] - w_int[1][0]
    v_shr = w_int[2][-1] - w_int[2][0]
    
    shr_mag = np.sqrt((u_shr ** 2) + (v_shr ** 2))
    
    return u_shr, v_shr, shr_mag

    

In [105]:
#Supercell composite calculation. Should use mucape, esrh, and ebwd, but for now we'll use sbcape, sfc-3 srh, and sfc-6 shear
def supercell_comp(mucape, esrh, ebwd):
    r"""Calculate the supercell composite parameter.
     
    The supercell composite parameter is designed to identify
    environments favorable for the development of supercells,
    and is calculated using the formula developed by 
    [Thompson, Edwards, and Mead, 2004]:
    
    SCP = (mucape / 1000 J/kg) * (esrh / 50 m^2/s^2) * (ebwd / 20 m/s) 
    
    The ebwd term is set to zero below 10 m/s and
    capped at 1 when ebwd exceeds 20 m/s.
    
    Parameters
    ----------
    mucape : array-like
        Most-unstable CAPE
    esrh : array-like
        Effective-layer storm-relative helicity
    ebwd : array-like
        Effective bulk shear
        
    Returns
    -------
    number
        supercell composite
        
    Citation:
    Thompson, R.L., R. Edwards, and C. M. Mead, 2004b:
        An update to the supercell composite
        and significant tornado parameters.
        Preprints, 22nd Conf. on Severe Local
        Storms, Hyannis, MA, Amer.
        Meteor. Soc.
        
    """
    
    ebwd = ebwd.to('m/s')
    ebwd = ebwd.magnitude
    inds = np.where((ebwd <= 20.) & (ebwd >= 10.))
    inds1 = np.where(ebwd < 10.)
    inds2 = np.where(ebwd > 20.)
    ebwd[inds] = ebwd[inds] / 20.
    ebwd[inds1] = 0.
    ebwd[inds2] = 1.
    sup_comp = (mucape.magnitude / 1000.) * (esrh.magnitude / 50.) * ebwd
     
    return sup_comp

In [106]:
def sigtor(sbcape, sblcl, srh1, shr6):
    r"""Calculate the significant tornado parameter (fixed layer).
     
    The significant tornado parameter is designed to identify
    environments favorable for the production of significant
    tornadoes contingent upon the development of supercells.
    It's calculated according to the formula used on the SPC
    mesoanalysis page, updated in [Thompson, Edwards, and Mead, 2004]:
    
    sigtor = (sbcape / 1500 J/kg) * ((2000 m - sblcl) / 1000 m) * (srh1 / 150 m^s/s^2) * (shr6 / 20 m/s)
    
    The sblcl term is set to zero when the lcl is above 2000m and 
    capped at 1 when below 1000m, and the shr6 term is set to 0 
    when shr6 is below 12.5 m/s and maxed out at 1.5 when shr6
    exceeds 30 m/s.
    
    Parameters
    ----------
    sbcape : array-like
        Surface-based CAPE
    sblcl : array-like
        Surface-based lifted condensation level
    srh1 : array-like
        Surface-1km storm-relative helicity
    shr6 : array-like
        Surface-6km bulk shear
        
    Returns
    -------
    number
        significant tornado parameter
    
    Citation:
    Thompson, R.L., R. Edwards, and C. M. Mead, 2004b:
        An update to the supercell composite
        and significant tornado parameters.
        Preprints, 22nd Conf. on Severe Local
        Storms, Hyannis, MA, Amer.
        Meteor. Soc.
        
    """
    
    shr6 = shr6.to('m/s')
    shr6 = shr6.magnitude
    sblcl = sblcl.magnitude
    ind = np.where((sblcl <= 2000.) & (sblcl >= 1000.))
    ind1 = np.where(sblcl < 1000.)
    ind2 = np.where(sblcl > 2000.)
    sind = np.where((shr6 <= 30.) & (shr6 >= 12.5))
    sind1 = np.where(shr6 < 12.5)
    sind2 = np.where(shr6 > 30.)
    sblcl[ind] = (2000. - sblcl[ind]) / 1000.
    sblcl[ind1] = 1.
    sblcl[ind2] = 0.
    shr6[sind] = shr6[sind] / 20.
    shr6[sind1] = 0.
    shr6[sind2] = 1.5
    sigtor = (sbcape.magnitude / 1500.) * sblcl * (srh1.magnitude / 150.) * shr6
     
    return sigtor

In [107]:
#Let's try to calculate the significant hail parameter.
def significant_hail(pressure, u, v, height, temperature, dewpoint):
    r"""Calculate the Significant Hail Parameter (SHIP)
    """
    

In [108]:
def mean_pressure_weighted(pressure, *args, **kwargs):
    r"""Calculate pressure-weighted mean of an arbitrary variable through a layer.

    Layer top and bottom specified in height or pressure.

    Parameters
    ----------
    pressure : `pint.Quantity`
        Atmospheric pressure profile
    *args : `pint.Quantity`
        Parameters for which the pressure-weighted mean is to be calculated.
    heights : `pint.Quantity`, optional
        Heights from sounding
    bottom: `pint.Quantity`, optional
        The bottom of the layer in either the provided height coordinate
        or in pressure. Don't provide in meters AGL unless the provided
        height coordinate is meters AGL. Default is the first observation,
        assumed to be the surface.
    depth: `pint.Quantity`, optional
        The depth of the layer in meters or hPa.

    Returns
    -------
    `pint.Quantity`
        u_mean: u-component of layer mean wind.
    `pint.Quantity`
        v_mean: v-component of layer mean wind.

    """
    heights = kwargs.pop('heights', None)
    bottom = kwargs.pop('bottom', None)
    depth = kwargs.pop('depth', None)
    ret = []  # Returned variable means in layer
    for datavar in args:
        layer_p, layer_arg = get_layer(pressure, datavar, heights=heights,
                                       bottom=bottom, depth=depth)
        # Taking the integral of the weights (pressure) to feed into the weighting
        # function. Said integral works out to this function:
        pres_int = 0.5 * (layer_p[-1].magnitude**2 - layer_p[0].magnitude**2)
        arg_mean = (np.trapz(layer_arg * layer_p, x=layer_p) / pres_int)
        ret.append(arg_mean * datavar.units)

    return ret

In [109]:
def mean_wind_pressure_weighted_old(u, v, p, hgt=None, depth=None, bottom=None, obs_only=False):
    r"""Calculate pressure-weighted mean wind through a layer. 
    
    Layer top and bottom specified in height or pressure.
    
    Parameters
    ----------
    u : `pint.Quantity`
        U-component of wind.
    v : `pint.Quantity`
        V-component of wind.
    p : `pint.Quantity`
        Atmospheric pressure profile
    hgt : `pint.Quantity`
        Heights from sounding
    depth: `pint.Quantity`
        The depth of the layer in meters or hPa.
    bottom: `pint.Quantity`
        The bottom of the layer in either the provided height coordinate
        or in pressure. Don't provide in meters AGL unless the provided
        height coordinate is meters AGL. Default is the first observation,
        assumed to be the surface.
        
    Returns
    -------
    `pint.Quantity`
        u_mean: u-component of layer mean wind, in m/s
    `pint.Quantity`
        v_mean: v-component of layer mean wind, in m/s

    """  
    u = u.to('meters/second')
    v = v.to('meters/second')
    layer_p, layer_u, layer_v = get_layer(p, u, v, heights=hgt, bottom=bottom, depth=depth)
    
    u_mean = np.trapz(layer_u * layer_p, x=layer_p) / np.trapz(layer_p, x=layer_p) * units('m/s')
    v_mean = np.trapz(layer_v * layer_p, x=layer_p) / np.trapz(layer_p, x=layer_p) * units('m/s')

    return u_mean, v_mean
    

In [110]:
def critical_angle_old(ushr, vshr, um, vm, usfc, vsfc):
    r"""Calculate the critical angle.
    
    The critical angle is the angle between the 10m storm-relative inflow vector
    and the 10m-500m shear vector. A critical angle near 90 degrees indicates 
    that a storm in this environment on the indicated storm motion vector 
    is likely ingesting purely streamwise vorticity into its updraft, and Esterheld and 
    Giuliana (2008) showed that significantly tornadic supercells tend to occur in environments
    with critical angles near 90 degrees.

    Parameters
    ----------
    
    ushr : array-like
        U-component of 10m-500m shear vector.
    vshr : array-like
        V-component of 10m-500m shear vector.
    um : array-like
        U-component of storm motion.
    vm : array-like
        V-component of storm motion.
    usfc : array-like
        10m wind u component.
    vsfc :
        10m wind v component.
        
    Returns
    -------
    `pint.Quantity'
        critical angle in degrees
        
    
    Esterheld, J., & Giuliano, D. (2008). Discriminating between Tornadic and Non-Tornadic Supercells:
    A New Hodograph Technique. E-Journal Of Severe Storms Meteorology, 3(2).Retrieved July 10, 2017, 
    from http://ejssm.org/ojs/index.php/ejssm/article/view/33/38"""  
    
    #Make everything relative to the sfc wind orientation
    umn = um - usfc
    vmn = vm - vsfc
    
    #Get direction of each vector
    shrdir = get_wind_dir(ushr, vshr)
    smdir = get_wind_dir(umn,vmn)
    #Account for really weird, unlikely storm motions to the SW
    smdir[(smdir.magnitude < shrdir.magnitude) & (smdir.magnitude < 90.)] = smdir[(smdir.magnitude < shrdir.magnitude) & (smdir.magnitude < 90.)] + (360. * units('degrees')) 
    critical_angle = smdir - shrdir
    return critical_angle

In [123]:
def critical_angle(pressure, u, v, heights, stormu, stormv):
    r"""Calculate the critical angle.
    
    The critical angle is the angle between the 10m storm-relative inflow vector
    and the 10m-500m shear vector. A critical angle near 90 degrees indicates 
    that a storm in this environment on the indicated storm motion vector 
    is likely ingesting purely streamwise vorticity into its updraft, and Esterheld and 
    Giuliana (2008) showed that significantly tornadic supercells tend to occur in environments
    with critical angles near 90 degrees.

    Parameters
    ----------
    
    pressure : array-like
        Pressures from sounding.
    u : array-like
        U-component of sounding winds.
    v : array-like
        V-component of sounding winds.
    heights : array-like
        Heights from sounding.
    stormu : array-like
        U-component of storm motion.
    stormv :
        V-component of storm motion.
        
    Returns
    -------
    `pint.Quantity'
        critical angle in degrees
        
    
    Esterheld, J., & Giuliano, D. (2008). Discriminating between Tornadic and Non-Tornadic Supercells:
    A New Hodograph Technique. E-Journal Of Severe Storms Meteorology, 3(2).Retrieved July 10, 2017, 
    from http://ejssm.org/ojs/index.php/ejssm/article/view/33/38""" 
    
    #Convert everything to m/s
    u = u.to('m/s')
    v = v.to('m/s')
    stormu = stormu.to('m/s')
    stormv = stormv.to('m/s')
    
    sort_inds = np.argsort(pressure[::-1])
    pressure = pressure[sort_inds]
    heights = heights[sort_inds]
    u = u[sort_inds]
    v = v[sort_inds]

    #Calculate sfc-500m shear vector
    shr5 = bulk_shear(pressure, u, v, heights=heights, depth = 500 * units('meter'))
    
    #Make everything relative to the sfc wind orientation
    umn = stormu - u[0]
    vmn = stormv - v[0]
    
    vshr = np.asarray([shr5[0].magnitude, shr5[1].magnitude])
    vsm = np.asarray([umn.magnitude, vmn.magnitude])
    angle_c = np.dot(vshr,vsm)/np.linalg.norm(vshr)/np.linalg.norm(vsm)
    critical_angle = np.arccos(np.clip(angle_c, -1, 1))
    
    return (critical_angle * units('radian')).to('degrees')

In [124]:
def energy_helicity_index(cape, srh):
    """Calculate the energy-helicity index (EHI).
    
    Requires CAPE and storm-relative helicity (either sfc-1km, sfc-3km, or effective).
    
    Parameters
    ----------
    
    cape : array-like
        Convective Available Potential Energy.
    srh : array-like
        Storm-relative helicity over the desired layer.
      
    Returns
    -------
    `pint.Quantity'
        Energy-helicity index (dimensionless)
        
    Citation:
    Rasmusen, E. & Blanchard, D. (1998). A Baseline Climatology of Sounding-Derived Supercell and
    Tornado Forecast Parameters. Wea. Forecasting., 13, 1148-1164,
    doi: https://doi.org/10.1175/1520-0434(1998)013<1148:ABCOSD>2.0.CO;2
    
    """
    
    ehi = (cape.magnitude * srh.magnitude) / 160000
    
    return ehi

In [125]:
def test_ehi():
    """Test energy helicity index function."""
    cape = [1000., 1000., 3000., 1000., 3000, 6000] * units('J/kg')
    srh = [100., 200., 200., 600., 300, 400] * units('m^2/s^2')
    truth = [0.625, 1.25, 3.75, 3.75, 5.625, 15]
    ehi = energy_helicity_index(cape, srh)
    assert_almost_equal(ehi, truth, 6)    

In [126]:
test_ehi()

In [127]:
def test_bulk_shear():
    """Test bulk shear with observed sounding."""
    with UseSampleData():
        data = get_upper_air_data(datetime(2016, 5, 22, 0), 'DDC', source='wyoming')
    shr6 = bulk_shear(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], heights=data.variables['height'][:], depth = 6000 * units('meter'))
    truth = [29.899581266946115, -14.389225800205509, 33.181844117951236] * units('knots')
    assert_array_equal(shr6[0].to('knots'), truth[0])    
    assert_array_equal(shr6[1].to('knots'), truth[1])    
    assert_array_equal(shr6[2].to('knots'), truth[2])

In [128]:
def test_bulk_shear_elevated():
    """Test bulk shear with observed sounding and a base above the surface."""
    with UseSampleData():
        data = get_upper_air_data(datetime(2016, 5, 22, 0), 'DDC', source='wyoming')
    shr6 = bulk_shear(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], heights=data.variables['height'][:], bottom = data.variables['height'][0] + 3000 * units('meter'), depth=3000 * units('meter'),)
    truth = [0.9655943923302139, -3.8405428777944466, 3.9600684497464442] * units('m/s')
    assert_array_equal(shr6[0], truth[0])    
    assert_array_equal(shr6[1], truth[1])    
    assert_array_equal(shr6[2], truth[2])

In [129]:
test_bulk_shear()

In [130]:
test_bulk_shear_elevated()

In [131]:
def test_critical_angle():
    """Test critical angle with observed sounding."""
    with UseSampleData():
        data = get_upper_air_data(datetime(2016, 5, 22, 0), 'DDC', source='wyoming')
    critical = critical_angle(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], data.variables['height'][:], stormu = 20 * units('knot'), stormv = 20 * units('knot')) 
    truth = [73.5228229035071] * units('degree')
    assert_array_equal(critical, truth)    

In [132]:
test_critical_angle()

In [ ]:
def test_bunkers_motion():
    """Test Bunkers storm motion with observed sounding."""
    with UseSampleData():
        data = get_upper_air_data(datetime(2016, 5, 22, 0), 'DDC', source='wyoming')
    motion = bunkers_storm_motion(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], 
                                  data.variables['height'][:])
    truth = [1.4537892577864744, 2.0169333025630616, 10.587950761120482, 13.915130377372801, 6.0208700094534775, 7.9660318399679308] * units('m/s')
    assert_almost_equal(motion[0][0], truth[0], 8)    
    assert_almost_equal(motion[0][1], truth[1], 8)    
    assert_almost_equal(motion[1][0], truth[2], 8)
    assert_almost_equal(motion[1][1], truth[3], 8)    
    assert_almost_equal(motion[2][0], truth[4], 8)    
    assert_almost_equal(motion[2][1], truth[5], 8)                              
                                  

In [ ]:
test_bunkers_2000()

In [ ]:
def test_supercell_composite():
    """Test supercell composite function."""
    mucape = [2000., 1000., 500., 2000.] * units('J/kg')
    esrh = [400., 150., 45., 45.] * units('m^2/s^2')
    ebwd = [30., 15., 5., 5.] * units('m/s')
    truth = [16., 2.25, 0., 0.]
    supercell_composite = supercell_comp(mucape, esrh, ebwd)
    assert_array_equal(supercell_composite, truth)    
                             
                                  

In [ ]:
test_supercell_composite()

In [ ]:
def test_sigtor():
    """Test significant tornado parameter function."""
    sbcape = [2000., 2000., 2000., 2000., 3000, 4000] * units('J/kg')
    sblcl = [3000., 1500., 500., 1500., 1500, 800] * units('meter')
    srh1 = [200., 200., 200., 200., 300, 400] * units('m^2/s^2')
    shr6 = [20., 5., 20., 35., 20., 35] * units('m/s')
    truth = [0., 0, 1.777778, 1.333333, 2., 10.666667]
    significant_tornado = sigtor(sbcape, sblcl, srh1, shr6)
    assert_almost_equal(significant_tornado, truth, 6)    
                             

In [ ]:
test_sigtor()

In [ ]:
def test_mean_wind_pressure_weighted():
    """Test pressure-weighted mean wind function with vertical interpolation."""
    with UseSampleData():
        data = get_upper_air_data(datetime(2016, 5, 22, 0), 'DDC', source='wyoming')
    mean6 = mean_pressure_weighted(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], heights=data.variables['height'][:], depth = 6000 * units('meter'))
    truth = [6.0208700094534775, 7.966031839967931] * units('m/s')
    assert_almost_equal(mean6[0], truth[0], 7)    
    assert_almost_equal(mean6[1], truth[1], 7)    

In [ ]:
test_mean_wind_pressure_weighted()

In [ ]:
mean6 = mean_pressure_weighted(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], height=data.variables['height'][:], depth = 3000 * units('meter'), bottom = 3000 * units('meter'))
print(mean6)

In [ ]:
def test_mean_wind_pressure_weighted_elevated():
    """Test pressure-weighted mean wind function with a base above the surface."""
    with UseSampleData():
        data = get_upper_air_data(datetime(2016, 5, 22, 0), 'DDC', source='wyoming')
    mean6 = mean_pressure_weighted(data.variables['pressure'][:], data.variables['u_wind'][:], data.variables['v_wind'][:], heights=data.variables['height'][:], depth = 3000 * units('meter'), bottom = data.variables['height'][0] + 3000 * units('meter'))
    truth = [8.270829843626476, 1.7392601775853547] * units('m/s')
    assert_almost_equal(mean6[0], truth[0], 7)    
    assert_almost_equal(mean6[1], truth[1], 7)    

In [ ]:
test_mean_wind_pressure_weighted_elevated()